In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime

In [6]:
from research.strings import TARGET, PREDICTORS, INSPECTORS
from research.evaluation import evaluate_model

In [3]:
d_test = pd.read_csv("data/test_data.csv")
y_test = d_test[TARGET]
X_test = d_test[PREDICTORS]
clf = pickle.load(open("models/sklearn-glm.pkl", "rb"))
evaluate_model(y_test, clf.predict(X_test))

F1 Score = 0.08664
Precision = 0.63158
Recall = 0.04651


,Predicted +,Predicted -
Actual +,12,246
Actual -,7,1372


In [8]:
d_test["inspector"] = d_test[INSPECTORS].idxmax(axis=1).apply(lambda x: str(x)[10:])
d_test.head()

,criticalFound,Inspection_Date,License,Inspection_ID,Business_ID,Inspector_blue,Inspector_brown,Inspector_green,Inspector_orange,Inspector_purple,...,pastCritical,timeSinceLast,ageAtInspection,consumption_on_premises_incidental_activity,tobacco_retail_over_counter,temperatureMax,heat_burglary,heat_sanitation,heat_garbage,inspector
0,0.0,16324,2021618,920471,2021618-20121216,1,0,0,0,0,...,0.0,2.000000,1,0.0,0.0,77.946667,70.000000,44.398934,18.826366,blue
1,0.0,16339,43618,1115257,43618-20140616,0,0,0,1,0,...,0.0,1.917808,1,0.0,0.0,72.263333,25.084509,52.785171,11.752896,orange
2,0.0,16346,67839,1155245,67839-20121116,0,0,0,1,0,...,0.0,1.936986,1,0.0,0.0,65.810000,12.395546,13.437666,41.032810,orange
3,0.0,16364,2183764,1155246,2183764-20130416,0,0,0,1,0,...,0.0,1.115068,0,0.0,0.0,55.386667,6.591129,12.044724,6.493034,orange
4,1.0,16316,2008126,1199491,2008126-20130516,1,0,0,0,0,...,1.0,1.342466,1,0.0,0.0,82.030000,9.700997,14.368083,0.453519,blue


In [11]:
d_test.groupby(["inspector", "pastCritical"])["criticalFound"].count()

inspector  pastCritical
blue       0.0             480
           1.0              57
brown      0.0             104
           1.0              11
green      0.0             319
           1.0              26
orange     0.0             272
           1.0              27
purple     0.0              84
           1.0              15
yellow     0.0             220
           1.0              22
Name: criticalFound, dtype: int64

In [12]:
d_test.groupby(["inspector", "pastCritical"])["criticalFound"].mean()

inspector  pastCritical
blue       0.0             0.187500
           1.0             0.298246
brown      0.0             0.019231
           1.0             0.090909
green      0.0             0.109718
           1.0             0.153846
orange     0.0             0.154412
           1.0             0.148148
purple     0.0             0.369048
           1.0             0.800000
yellow     0.0             0.077273
           1.0             0.136364
Name: criticalFound, dtype: float64